In [1]:
#cmd: cd desktop/Git/DataScience/covid-19
#activate py38
#jupyter notebook

In [2]:
from datetime import datetime as dt
from pathlib import Path as path
from ambiente import DSAmbiente

In [3]:
from matplotlib import pyplot
import seaborn as sb
import pandas as pd

In [4]:
from meta import Meta
meta = Meta()
meta.set_meta('__datetime__', dt(2020,6,16,9,53))
meta.set_meta('__credits__',['CWOJR'])
meta.set_meta('__author__', 'jrCleber')

In [5]:
#instanciando objeto
ab = DSAmbiente()

In [6]:
#alocando path do dataset do ambiente de trabalhoe em uma variável
covid_xlsx = ab.file_data_ambiente()
covid_xlsx

SELECIONE O DATASET:
____________________________________________________________
C:\Users\clebe\Desktop\Python\Analises\002 - Dados Preparados
(0) para para o Dataset: hist_covid19br_2020-06-15.xlsx
____________________________________________________________
------------------------------------------------------------
Escolha: 0


WindowsPath('C:/Users/clebe/Desktop/Python/Analises/002 - Dados Preparados/hist_covid19br_2020-06-15.xlsx')

# Lendo arquiv hist_covid19br_2020-06-15.xlsx com pandas

origem: https://covid.saude.gov.br/

In [7]:
covid_dataframe = pd.read_excel(covid_xlsx)
covid_dataframe.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125,0,0,0,0,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125,1,1,0,0,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125,1,0,0,0,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125,1,0,0,0,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125,2,1,0,0,NaN,NaN


In [8]:
#sem alocar o path em uma variável
#covid_dataframe = pd.read_excel(ab.file_data_ambiente())

In [9]:
#linhas nulas (NaN)
covid_dataframe.isnull().sum()

regiao                        0
estado                      112
municipio                  4675
coduf                         0
codmun                     3136
codRegiaoSaude             4675
nomeRegiaoSaude            4675
data                          0
semanaEpi                     0
populacaoTCU2019           1539
casosAcumulado                0
casosNovos                    0
obitosAcumulado               0
obitosNovos                   0
Recuperadosnovos         389934
emAcompanhamentoNovos    389934
dtype: int64

In [10]:
#enviando para analises as linhas com campos em branco da coluna municipio
covid_isnull = covid_dataframe[covid_dataframe.municipio.isnull()]
analise = path.home() / 'Desktop' / 'Python' / 'Analises' / '003 - Analises'
name_null = 'null-lines-remove-from-hist_covid19br_2020-06-15.xlsx'
covid_isnull.to_excel(analise/name_null, encoding='latin-1', index=False)

In [11]:
#Removendo linhas onde o município é nullo
covid_file = covid_dataframe[covid_dataframe.municipio.notnull()]

In [12]:
covid_file.isnull().sum()

regiao                        0
estado                        0
municipio                     0
coduf                         0
codmun                        0
codRegiaoSaude                0
nomeRegiaoSaude               0
data                          0
semanaEpi                     0
populacaoTCU2019              0
casosAcumulado                0
casosNovos                    0
obitosAcumulado               0
obitosNovos                   0
Recuperadosnovos         385317
emAcompanhamentoNovos    385317
dtype: int64

In [13]:
#Selecionando colunas com as quais não iremos trabalhar
colunas = ['regiao', 'estado', 'municipio', 'nomeRegiaoSaude', 'data', 'semanaEpi', 'casosNovos', 'obitosNovos', 'populacaoTCU2019']
covid_work = covid_file.loc[:, colunas]

In [14]:
covid_work.head()

,regiao,estado,municipio,nomeRegiaoSaude,data,semanaEpi,casosNovos,obitosNovos,populacaoTCU2019
3217,Norte,RO,Alta Floresta D'Oeste,ZONA DA MATA,2020-03-27,13,0,0,22945
3218,Norte,RO,Alta Floresta D'Oeste,ZONA DA MATA,2020-03-28,13,0,0,22945
3219,Norte,RO,Alta Floresta D'Oeste,ZONA DA MATA,2020-03-29,14,0,0,22945
3220,Norte,RO,Alta Floresta D'Oeste,ZONA DA MATA,2020-03-30,14,0,0,22945
3221,Norte,RO,Alta Floresta D'Oeste,ZONA DA MATA,2020-03-31,14,0,0,22945


In [15]:
covid_work.dtypes

regiao                      object
estado                      object
municipio                   object
nomeRegiaoSaude             object
data                datetime64[ns]
semanaEpi                    int64
casosNovos                   int64
obitosNovos                  int64
populacaoTCU2019            object
dtype: object

# Convertendo populacaoTCU2019 em int 

In [16]:
#escapando o (.) ponto e o removendo
covid_work.populacaoTCU2019 = covid_work.populacaoTCU2019.str.replace(r"\.", "")

In [17]:
#escapando o ( ) espaço e o removendo
covid_work.populacaoTCU2019 = covid_work.populacaoTCU2019.str.replace(r"\ ", "")

In [18]:
#removendo caracteres implementados pelo TCU na string 
populacaoTCU2019_int = []
for populacao in covid_work.populacaoTCU2019:
    pop_list = list(populacao)
    i = 0
    for x in range(len(pop_list)):
        if pop_list[x] == "(":
            i = x
    if "(" in pop_list:
        del(pop_list[i:])
    pop = ''
    for x in range(len(pop_list)):
        pop = pop + pop_list[x]
    #Convertendo para inteiro
    populacaoTCU2019_int.append(int(pop))

In [19]:
type(populacaoTCU2019_int[0])

int

In [20]:
#adicionando a coluna convertida no dataframe
covid_work['populacaoTCU2019_int'] = pd.Series(populacaoTCU2019_int)

In [21]:
covid_work.dtypes

regiao                          object
estado                          object
municipio                       object
nomeRegiaoSaude                 object
data                    datetime64[ns]
semanaEpi                        int64
casosNovos                       int64
obitosNovos                      int64
populacaoTCU2019                object
populacaoTCU2019_int           float64
dtype: object

In [22]:
#Deletado coluna 'populacaoTCU2019' do tipo 'object' do DataFrame
covid_work.drop('populacaoTCU2019', axis=1, inplace=True)

In [23]:
covid_work.head()

,regiao,estado,municipio,nomeRegiaoSaude,data,semanaEpi,casosNovos,obitosNovos,populacaoTCU2019_int
3217,Norte,RO,Alta Floresta D'Oeste,ZONA DA MATA,2020-03-27,13,0,0,15852.0
3218,Norte,RO,Alta Floresta D'Oeste,ZONA DA MATA,2020-03-28,13,0,0,15852.0
3219,Norte,RO,Alta Floresta D'Oeste,ZONA DA MATA,2020-03-29,14,0,0,15852.0
3220,Norte,RO,Alta Floresta D'Oeste,ZONA DA MATA,2020-03-30,14,0,0,15852.0
3221,Norte,RO,Alta Floresta D'Oeste,ZONA DA MATA,2020-03-31,14,0,0,15852.0


In [24]:
#exportando Dataframe para dados preparados
preparados = path.home() / 'Desktop' / 'Python' / 'Analises' / '002 - Dados Preparados'
name_work = 'covid_work_2020-06-16.xlsx'
covid_work.to_excel(preparados/name_work, encoding='latin-1', index=False)